<a href="https://colab.research.google.com/github/malanim/text_generator/blob/main/text-generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Char-based text generation with LSTM

In [2]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [4]:
TRAIN_TEXT_FILE_PATH = 'train_text.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])

    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'е', 'а', 'и', 'т', 'н', 'с', 'р', 'в', 'л', 'к', 'м', 'д', 'у', 'п', ',', 'я', 'ы', 'ь', 'б', 'г', 'з', 'ч', '.', 'й', 'ж', 'х', 'ю', 'ш', '\n', 'ц', 'э', 'щ', 'Р', '—', 'В', 'С', 'Н', 'ф', 'М', 'ё', '-', 'И', 'П', 'К', '–', 'У', 'Е', 'А', ':', '?', '«', '»', 'Т', 'Я', 'Э', 'О', 'Д', '…', '(', ')', 'З', 'Ч', 'Б', '0', 'Ш', '!', 'ъ', 'Ф', 'Г', 'Л', 'Х', '2', '1', '‑', '9', '”', '“', '5', '4', 'Ц', '3', 'i', 'Ю', '8', 'Ж', '\xad', 'L', 'a', 'm', 'b', 'o', 'r', 'g', 'h', 'n', '\u200e', '7', 'C', ';']


In [5]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [6]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text

    _, hidden = model(train, hidden)

    inp = train[-1].view(-1, 1, 1)

    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char

    return predicted_text

In [7]:
class TextRNN(nn.Module):

    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)

    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)

    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    patience=5,
    verbose=True,
    factor=0.5
)

n_epochs = 100000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Выходные данные были обрезаны до нескольких последних строк (5000).
 просто нас кого нас, предательство напрягаюсь».
 Самое плавания Конституционный захват власти, кровавый с жертвами, довели дело до войны на юго-востоке Украины, Крым объявил о своей независимости и пр
Loss: 0.18770653754472733
 просто нас кого насто не вострегите, что Россия может подняться с колен и как следует огреть.
 Ну вышлем мы этих разведчиков, приедут другие, может быть, умных пришлют, намучаемся потом искать их. Мы 
Loss: 0.18718259334564208
 правилах вперёд, мики мы не пойти в состоянии легко отличить желание к новизне, к обновлению от политических провокаций, которые ставят перед собой только одну цель – развалить российскую государствен
Loss: 0.1885777407884598
 просто нас кого нас, предательство народа, в наших людях, в наших традициях и нашей культуре, в нашей экономике, в огромной нашей территории и природных богатствах, в обороноспособности, конечно. Но с
Loss: 0.18508541017770766
 просто нас кого нас,

In [15]:
model.eval()

print(evaluate(
    model,
    char_to_idx,
    idx_to_char,
    temp=0.3,
    prediction_len=1000,
    start_text='. '
    )
)

. махать, что они сочет нефони накоторым.
 Если возможно, нет никаких ограничений.
 То, что сейчас США вышли на первые позиции в разрешении этих (украинских) кризисных явлений, говорит о том, что они вначале этот процесс развивали.
 Я не стремился командовать. Важнее было сохранить независимость.
 Ввергли страну в хаос, в развал экономики и социальной сферы, в гражданскую войну с огромными жертвами. Зачем? Видимо те, кто ляпает цветные революции, считает себя гениальными художниками и никак не могут остановиться.
 Эту беду пытаются использовать еще и недобросовестным образом. Пытаются, я бы сказал даже грубее, раздувать какие-то политические жизни. Вот о собираю в отношения с колен к все равно не должен иметь голову. И, чтобы выстраивать межгосударственные отношения, нужно руководствоваться не эмоциями, а фундаментальными интересами своих стран.
 Если вам не нравятся мои ответы, то вы тогда не задавайте вопросов.
 Если весь бонус России заключается в том, что ей разрешают рядом посидет